In [1]:
import os
import sys

import json
import re
import torch

from sentence_transformers import SentenceTransformer

sys.path.append("../")
sys.path.append("../loaders/")
from rag_utils import RAG_Utils

from config import Config
cfg = Config("uk")

rag = RAG_Utils(cfg)

# from qdrant_utils import Qdrant
# qdrant = Qdrant(cfg.QDRANT_URL, cfg.QDRANT_WEBSITES, SentenceTransformer(cfg.SENTENCE_TRANSFORMER))
qdrant = rag.get_qdrant_utils()

using the following configuration:

The Config objects offers the following variables
------ WEBSITE STORAGE ----------
WEBSITE_STORAGE_PATH    /local/innecs-local/uk-website-data

------ QDRANT VECTOR STORE ------
QDRANT_URL url:         http://qdrant:6333
QDRANT_WEBSITES:      ies_uk_website_sentences
QDRANT_WEBSITES_SUMMARIES:      ies_uk_website_summaries

------ JSON_DATA ----------------
JSON_FILE_PATH:         /local/innecs-local/uk-website-data.json
JSON_COMPANY_ID:        company_id
JSON_COMPANY_NAME:      company_name
JSON_COMPANY_URL:      weburl

------ SQLITE -------------------
SQLITE_FILEPATH:        /local/innecs-local/db/rag-tasks-uk.db

------ NLP ----------------------
SPACY_MODEL:            en_core_web_lg
SENTENCE_TRANSFORMER:   all-MiniLM-L6-v2
LLM:                    meta-llama/Meta-Llama-3.1-8B-Instruct



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0
/home/dev/git/ies/retrieval-augmented-exploration/website-rag/playground/../loaders/qdrant_utils.py:11: UserWarning: Qdrant client version 1.14.2 is incompatible with server version 1.12.6. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  self.qdrant = QdrantClient(url, timeout=600000)


In [3]:
data = qdrant.get_company_data("12104755")
data

{'company_name': 'NEPTUNO MANAGEMENT LTD',
 'company_id': '12104755',
 'weburl': 'http://www.bestinternationalgroup.com',
 'category': 'Private Limited Company',
 'status': 'Active',
 'incorporation_date': '15/07/2019',
 'country_of_origin': 'United Kingdom',
 'address_posttown': 'KNUTSFORD',
 'address_postcode': 'WA16 8RB',
 'address_country': 'ENGLAND',
 'address_lines': ['MOSELEY HALL FARM', 'CHELFORD ROAD'],
 'sic_code_texts': ['82990 - Other business support service activities n.e.c.'],
 'storage_folder': '12104755_bestinternationalgroup-com'}

In [11]:
def query(query):
    query = query
    similar_sents = rag._find_similar(query)        

    scored_answers = []
    for company_id in tqdm(similar_sents.keys()):
        company_data = similar_sents[company_id]
        print(company_data)
        break;
        company_name = ""
        web=""
        
        context = []
        for file in company_data.keys():
            company_name = company_data[file][0]["name"]
            web = company_data[file][0]["web"]
            indices = [entry["idx"] for entry in company_data[file]]                
            
            for (start_idx, end_idx) in self._create_augmented_company_context_windows(indices):
                context.append(self.qdrant.get_sentence_range(company_id, file, start_idx, end_idx))
    
            context.append("############") 
        
        a = self._answer(query, company_name, " ".join(list(context))).strip()        
        scored_answers.append({
            "score": a[0], 
            "name": company_name, 
            "website": web, 
            "explanation": a[2:].strip()
        })

    # return sorted(scored_answers, key=lambda x: x["score"], reverse=True)

query("")

{'company_name': 'NEPTUNO MANAGEMENT LTD',
 'company_id': '12104755',
 'weburl': 'http://www.bestinternationalgroup.com',
 'category': 'Private Limited Company',
 'status': 'Active',
 'incorporation_date': '15/07/2019',
 'country_of_origin': 'United Kingdom',
 'address_posttown': 'KNUTSFORD',
 'address_postcode': 'WA16 8RB',
 'address_country': 'ENGLAND',
 'address_lines': ['MOSELEY HALL FARM', 'CHELFORD ROAD'],
 'sic_code_texts': ['82990 - Other business support service activities n.e.c.'],
 'storage_folder': '12104755_bestinternationalgroup-com'}